# Design



### Challenge Name: fw_protect (/embsec/design/fw_protect)

The purpose of this tool will be to secure the firmware. Eventually,
you will want to keep the firmware confidential and be able to verify its integrity and authenticity. But for now, 
you will implement a firmware protect tool that does not add security. This tool should simply:

    1. Pack the version "0" into a little-endian short
    2. Load the firmware binary from firmware.bin
    3. Pack the size of the firmware into a little-endian short
    4. Append the release message "First version of firmware." to end of firmware (make sure it is null-terminated)
    5. Build and send a binary blob in the following format to the grader:
    
        [ 0x2 ]      [ 0x2 ]      [ variable ]           [ variable ]
        -------------------------------------------------------------------------
        version | firmware size |   firmware   |   release message + null-byte   




In [1]:
from embsec import Serial
import struct

def fw_protect():
    ser = Serial("/embsec/design/fw_protect")
    # Your code goes here!
    
    
    with open("firmware_0.bin", "rb") as f:
        file_contents = f.read() 
        
    size = len(file_contents)
    release_message = 'First version of firmware.'.encode() + b'\0'
    
    
    update = struct.pack('<hh', 0,size)
    firm_write = file_contents + release_message 
    
    
    ser.write(update)
    print("updated!")
    ser.write(firm_write)
    print("more updated!")
    
    return ser.read_until()

fw_protect()


updated!
more updated!


b'embsec{fw_protect_abf414be6977aaeb}\n'

### Challenge Name: fw_update (/embsec/design/fw_update)

For this lesson you will implement a basic update tool for the design challenge.
The tool will be used to update the device firmware. You should focus on reading in the firmware blob file, 
and constructing a set of frames to send to the bootloader. Pay attention to the structure of these 
frames, as the bootloader is expecting a certain size and format.

The bootloader is expecting frames of the following format:

      [ 0x2 ]    [ variable ]
    ----------------------------
    frame size |   data... 

Process:

1. Read in binary blob
2. Construct frames from blob
3. Send a 'U' for update mode & receive a confirmation 'U'
4. Send frames sequentially to boot loader



In [3]:
#we have to finish it off by sending a packet with size 0

#frame_size is a short


from embsec import Serial

def fw_update():
    ser = Serial("/embsec/design/fw_update")
    # Your code goes here!

    with open("firmware_1.blob", "rb") as f:
        file_contents = f.read() 
    print("file read in!")

    
    ser.write(b'U')
    print("confirmation message sent!")
    

    while ser.read() != b'U':
        pass
    print("u recieved!")
  

    ser.write(file_contents[:4])
    file_contents = file_contents[4:]
    print("metadata recieved!")
    
    
    for i in range(0, len(file_contents),32):
        while ser.read() != b'\0':
            pass
        
        file_chunk = file_contents[i:i+32]
        frame_size = len(file_chunk)
        frame = struct.pack('>h32s',frame_size, file_chunk)
        
        ser.write(frame)
        print("frame wrote!")
        
      
    end_frame = struct.pack('<h', 0)
    ser.write(end_frame)
    print("end frame sent!")
    return ser.read_until()

fw_update()


file read in!
confirmation message sent!
u recieved!
metadata recieved!
frame wrote!
frame wrote!
frame wrote!
end frame sent!


b'\x00embsec{fw_update_1e2e37bafe98506a}\n'